# Test Libreria Pyradise

In [1]:
import os 

os.chdir("/home/mario/NasPersonale/Progetti2/EUCAIM/")

In [2]:
import pyradise.fileio as fio 
import pyradise.process as proc
import pydicom

In [3]:
# import os
# import pydicom
# import numpy as np
# import SimpleITK as sitk
# from skimage.draw import polygon
# import highdicom as hd
# from datetime import datetime
# from highdicom.sr.coding import CodedConcept

# def read_dicom_series(series_dir):
#     """
#     Legge la serie DICOM usando SimpleITK e restituisce:
#       - l'immagine 3D (SimpleITK.Image)
#       - la lista dei nomi dei file della serie
#     """
#     reader = sitk.ImageSeriesReader()
#     series_ids = reader.GetGDCMSeriesIDs(series_dir)
#     if not series_ids:
#         raise ValueError("Nessuna serie DICOM trovata in " + series_dir)
#     series_file_names = reader.GetGDCMSeriesFileNames(series_dir, series_ids[0])
#     reader.SetFileNames(series_file_names)
#     image = reader.Execute()
#     return image, series_file_names

# def extract_roi_masks(rtstruct_path, sitk_image):
#     """
#     Legge il file RTSTRUCT (con pydicom) ed estrae, per ogni ROI, i contorni (in coordinate fisiche).
#     Converte i contorni in coordinate pixel usando sitk_image.TransformPhysicalPointToIndex
#     e rasterizza i poligoni con skimage.draw.polygon.
    
#     Restituisce un dizionario {ROI_name: mask} dove mask è un array numpy 3D con shape (depth, rows, cols).
#     """
#     ds_rt = pydicom.dcmread(rtstruct_path)
#     # Ottieni le dimensioni dell'immagine (SimpleITK restituisce (cols, rows, depth))
#     size = sitk_image.GetSize()  # (width, height, depth)
#     depth = size[2]
#     height = size[1]
#     width = size[0]
    
#     # Crea mapping da ROI number a ROI name, usando StructureSetROISequence
#     roi_num_to_name = {}
#     if hasattr(ds_rt, "StructureSetROISequence"):
#         for item in ds_rt.StructureSetROISequence:
#             roi_num_to_name[item.ROINumber] = item.ROIName
    
#     roi_masks = {}
#     if not hasattr(ds_rt, "ROIContourSequence"):
#         raise ValueError("RTSTRUCT non contiene ROIContourSequence!")
    
#     # Per ogni ROI nell'RTSTRUCT, crea una maschera vuota
#     for roi_item in ds_rt.ROIContourSequence:
#         roi_num = roi_item.ReferencedROINumber
#         roi_name = roi_num_to_name.get(roi_num, f"ROI_{roi_num}")
#         mask = np.zeros((depth, height, width), dtype=np.uint8)
#         # Controlla se esiste una ContourSequence
#         if hasattr(roi_item, "ContourSequence"):
#             for contour in roi_item.ContourSequence:
#                 if not hasattr(contour, "ContourData"):
#                     continue
#                 data = contour.ContourData  # Lista di float (x,y,z,...)
#                 pts = np.array(data).reshape(-1, 3)
#                 # Converte ogni punto fisico in indice (ritorna (col, row, slice))
#                 indices = [sitk_image.TransformPhysicalPointToIndex(tuple(pt)) for pt in pts]
#                 # Assumiamo che tutti i punti del contorno appartengano alla stessa slice:
#                 slice_idx = indices[0][2]
#                 # Estrai le coordinate in riga e colonna
#                 poly_cols = [ind[0] for ind in indices]
#                 poly_rows = [ind[1] for ind in indices]
#                 # Rasterizza il poligono
#                 rr, cc = polygon(poly_rows, poly_cols, shape=(height, width))
#                 mask[slice_idx, rr, cc] = 1
#         roi_masks[roi_name] = mask
#     return roi_masks

# def create_dicom_seg_with_sitk(dicom_series_path, rtstruct_path, output_seg_path):
#     """
#     Legge la serie DICOM e l'RTSTRUCT, estrae e rasterizza i contorni usando SimpleITK,
#     e crea un DICOM SEG multi-label usando highdicom.
#     """
#     # Legge la serie DICOM con SimpleITK
#     sitk_image, series_files = read_dicom_series(dicom_series_path)
#     print(f"✅ Serie DICOM letta con {len(series_files)} file.")
    
#     # Estrae le maschere ROI dal RTSTRUCT (registrate nello spazio dell'immagine)
#     roi_masks = extract_roi_masks(rtstruct_path, sitk_image)
#     if not roi_masks:
#         print("❌ Nessuna ROI estratta dall'RTSTRUCT.")
#         return
#     roi_names = list(roi_masks.keys())
#     print("✅ ROI estratte:", roi_names)
    
#     # Costruisce un array 4D (depth, height, width, num_ROI)
#     depth, height, width = roi_masks[roi_names[0]].shape
#     num_roi = len(roi_names)
#     seg_array = np.zeros((depth, height, width, num_roi), dtype=np.uint8)
#     for i, roi_name in enumerate(roi_names):
#         seg_array[..., i] = roi_masks[roi_name]
    
#     # Per il mapping dei colori: se non disponi dei dati di colore dall'RTSTRUCT,
#     # puoi definire una mappatura fissa oppure usare colori predefiniti.
#     # In questo esempio, usiamo colori predefiniti (uno diverso per ciascuna ROI).
#     default_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255),
#                       (255, 255, 0), (255, 0, 255), (0, 255, 255),
#                       (128, 128, 128)]
#     roi_colors = {name: default_colors[i % len(default_colors)] for i, name in enumerate(roi_names)}
    
#     # Crea le descrizioni dei segmenti usando highdicom
#     segment_descriptions = []
#     for i, name in enumerate(roi_names):
#         seg_desc = hd.seg.SegmentDescription(
#             segment_number=i + 1,
#             segment_label=name,
#             segmented_property_category=CodedConcept('T-D0050', 'SRT', 'Tissue'),
#             segmented_property_type=CodedConcept('M-03000', 'SRT', 'Tumor'),
#             # recommended_display_cielab_value=roi_colors[name],
#             algorithm_type=hd.seg.SegmentAlgorithmTypeValues.MANUAL
#         )
#         segment_descriptions.append(seg_desc)
    
#     now = datetime.now()
    
#     # Carica i file DICOM con pydicom per usarli come source_images
#     dicom_datasets = [pydicom.dcmread(f) for f in series_files]
    
#     seg_dataset = hd.seg.Segmentation(
#         source_images=dicom_datasets,
#         pixel_array=seg_array.astype(np.uint8),
#         segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
#         segment_descriptions=segment_descriptions,
#         series_instance_uid=hd.UID(),
#         series_number=1,
#         instance_number=1,
#         sop_instance_uid=hd.UID(),
#         manufacturer="MyCompany",
#         manufacturer_model_name="RTSTRUCT to SEG via SimpleITK",
#         software_versions="1.0",
#         series_description="RTSTRUCT to SEG (SimpleITK)",
#         # instance_creation_date=now.strftime("%Y%m%d"),
#         # instance_creation_time=now.strftime("%H%M%S.%f")[:-3],
#         # frame_of_reference_uid=dicom_datasets[0].FrameOfReferenceUID,
#         device_serial_number="123456"
#     )
    
#     pydicom.dcmwrite(output_seg_path, seg_dataset)
#     print("✅ DICOM SEG salvato in:", output_seg_path)

# # ESECUZIONE
# # Modifica i percorsi in base al tuo sistema


In [4]:
rtstruct_path = "DATA/RT-STRUCT/1-1.dcm"
dicom_series_path = "DATA/RT-STRUCT/DCM/ID_1/"
output_seg_path = "DATA/RT-STRUCT/seg.dcm"


In [5]:
# # create_dicom_seg(dicom_series_path,rtstruct_path, output_seg_path,force_frame_of_reference=True)
# create_dicom_seg_with_sitk(dicom_series_path,rtstruct_path, output_seg_path)


In [6]:
#Configuration for the notebook

%load_ext autoreload
%autoreload 2
import os 

os.chdir("/home/mario/NasPersonale/Progetti2/EUCAIM/")

In [7]:
from rtstruct.rtstruct2dcmseg import DicomSegmentationConverter

In [10]:

converter = DicomSegmentationConverter(
    dicom_series_path=dicom_series_path,
    rtstruct_path=rtstruct_path,
    output_seg_path=output_seg_path
)

converter.convert()

2025-03-04 17:20:52,216 - INFO - Starting DICOM series reading
2025-03-04 17:20:52,216 - INFO - Starting DICOM series reading
2025-03-04 17:20:52,216 - INFO - Starting DICOM series reading
2025-03-04 17:20:52,326 - INFO - Found DICOM series with ID: 1.3.6.1.4.1.14519.5.2.1.5168.2407.280813723089549264827522627812
2025-03-04 17:20:52,326 - INFO - Found DICOM series with ID: 1.3.6.1.4.1.14519.5.2.1.5168.2407.280813723089549264827522627812
2025-03-04 17:20:52,326 - INFO - Found DICOM series with ID: 1.3.6.1.4.1.14519.5.2.1.5168.2407.280813723089549264827522627812
ImageSeriesReader (0x6388bb819880): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0197711

2025-03-04 17:20:52,814 - INFO - Successfully read 91 DICOM files
2025-03-04 17:20:52,814 - INFO - Successfully read 91 DICOM files
2025-03-04 17:20:52,814 - INFO - Successfully read 91 DICOM files
2025-03-04 17:20:52,815 - INFO - Starting ROI mask extraction from RTSTRUCT
2025-03-04 17:20:52,815 - INFO - Startin